In [14]:
!pip install --upgrade transformers
!pip install transformers[torch]
!pip install 'accelerate>=0.26.0'


zsh:1: no matches found: transformers[torch]


In [15]:
import transformers
print(transformers.__version__)


4.51.3


In [16]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch

In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Cargar datos limpios
df = pd.read_csv("../../data/processed/train_preprocess_v1.csv")
df = df[['statement', 'label']]  # Asegurarse de usar solo lo necesario

# División
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(example):
    return tokenizer(example['statement'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# Eliminar columnas no necesarias
train_dataset = train_dataset.remove_columns(["statement", "__index_level_0__"])
val_dataset = val_dataset.remove_columns(["statement", "__index_level_0__"])



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Cellar/python@3.10/3.10.17/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.10/3.10.17/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/andrea/Documents/MUIRST/ABID/Reto2/bert_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File 

Map:   0%|          | 0/7160 [00:00<?, ? examples/s]

Map:   0%|          | 0/1790 [00:00<?, ? examples/s]

In [3]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from sklearn.metrics import f1_score

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=1).numpy()
    return {"f1": f1_score(labels, preds)}

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-test",
    num_train_epochs=1,                     
    per_device_train_batch_size=16,         
    per_device_eval_batch_size=32,          
    learning_rate=5e-5,                      
    weight_decay=0.01,
    eval_strategy="steps",            
    eval_steps=100,                   
    save_strategy="no",               
    logging_steps=50,                 
    disable_tqdm=False,               
    report_to="none"                        
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/_f/tnvd8_ms5rb8t4jqz9y92c2w0000gn/T/ipykernel_47439/242392240.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [4]:
trainer.train()
trainer.evaluate()

Step,Training Loss,Validation Loss


RuntimeError: Numpy is not available